In [1]:
!pip install tabpfn --no-index --find-links=file:////kaggle/input/tabpfn2/tabpfn-0.1.9-py3-none-any.whl


Looking in links: file://kaggle/input/tabpfn2/tabpfn-0.1.9-py3-none-any.whl
Processing //kaggle/input/tabpfn2/tabpfn-0.1.9-py3-none-any.whl


In [2]:
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff

!cp /kaggle/input/checkpoint/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

In [3]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
import xgboost
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier as lgb
import tabpfn
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)


In [4]:
df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')

greeks=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

times = greeks['Epsilon'].copy()

times[greeks['Epsilon'] != 'Unknown'] = greeks['Epsilon'][greeks['Epsilon'] != 'Unknown'].map(lambda x: datetime.strptime(x,'%m/%d/%Y').toordinal())

times[greeks['Epsilon'] == 'Unknown'] = np.nan

df['time']= times

df = df.dropna(subset = ['time']).reset_index(drop=True)

df.drop('Id', axis=1, inplace=True)
# df.drop('time', axis=1, inplace=True)

bad_cols=['AF', 'AH', 'AR', 'AX', 'BC', 'CF', 'CS', 'CU', 'CW ', 'DI', 'EE', 'EH', 'EJ', 'EL', 'FI','GI']

bad_cols=[ 'AR', 'AX', 'BC', 'CF', 'CS', 'CU', 'CW ', 'DI', 'EE', 'EH', 'EJ', 'EL', 'FI','GI']

train_df = df.drop('Class', axis=1)

train_df = train_df.drop(columns=bad_cols)

def remove_var(X_train, isVar, per = 0.1):
    # Convert all columns in X_train to numeric, coercing errors to NaN if any non-numeric data is present
    X_train_ = X_train.apply(pd.to_numeric, errors='coerce')
    
    # Calculate the variance of each column, ensuring that variances are in numeric format
    variances = X_train_.var().astype(float)
    
    # Sort variances in descending order and select the top percentage most variable columns
    top_var = int(len(variances) * per)
    
    high_variance_columns = variances.nlargest(top_var).index
    
    if not isVar:
        # Drop these high variance columns from the train_df
        X_train.drop(columns=high_variance_columns, inplace=True)
    else: 
        X_train = X_train[high_variance_columns]
    
    return X_train, high_variance_columns

# train_df, high_variance_columns = remove_var(train_df, isVar=False, per=0.1)

test_df = df['Class']
scale_pos = (test_df == 0).sum() / (test_df == 1).sum()

# change B values to 1 and A values to 0
# plt.figure(figsize=(15, 10))
# sns.boxplot(data=df, orient="h", palette="Set2")
# plt.title("Boxplots for All Columns")
# plt.xlabel("Values")
# plt.ylabel("Columns")
# plt.show()
#print(train_df.shape)

#train_df = train_df.dropna(subset=['time'])

#print(train_df.shape)


In [5]:
test_df_1 = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test_df_1.drop('Id', axis=1, inplace=True)
test_df_1 = test_df_1.apply(pd.to_numeric, errors='coerce')
test_df_1 = test_df_1.drop(columns=bad_cols)
test_df_1.fillna(test_df_1.mean(), inplace=True)
# test_df_1.drop(columns=high_variance_columns, inplace=True)

test_df_1['time'] = max(times)+1

# scale = StandardScaler().fit(test_df_1)

# test_df_1 = scale.transform(test_df_1)

print((test_df_1))


    AB   AF   AH   AM   AY   AZ  BD    BN   BP   BQ  ...   FE   FL   FR   FS  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    GB   GE   GF   GH   GL      time  
0  0.0  0.0  0.0  0.0  0.0  737698.0  
1  0.0  0.0  0.0  0.0  0.0  737698.0  
2  0.0  0.0  0.0  0.0  0.0  737698.0  
3  0.0  0.0  0.0  0.0  0.0  737698.0  
4  0.0  0.0  0.0  0.0  0.0  737698.0  

[5 rows x 43 columns]


In [6]:
# replace null values with the mean of the column
train_df = train_df.apply(pd.to_numeric, errors='coerce')

train_df.fillna(train_df.mean(), inplace=True)

scale = StandardScaler().fit(train_df)

train_df = scale.transform(train_df)

In [7]:
# function used to get score in competetion

def balanced_log_loss(y_true, y_pred):

    # Extracting class labels from y_true

    y_true = y_true.astype(int)

    

    # Computing the number of observations for each class

    N0 = np.sum(y_true == 0)

    N1 = np.sum(y_true == 1)

    

    # Calculating the inverse prevalence weights

    w0 = 1 / N0

    w1 = 1 / N1

    

    # Rescaling the predicted probabilities

    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    y_pred /= y_pred.sum(axis=1, keepdims=True)

    

    # Calculating the logarithmic loss for each class

    log_loss_0 = np.sum((1-y_true) * np.log(y_pred[:, 0])) / N0

    log_loss_1 = np.sum(y_true * np.log(y_pred[:, 1])) / N1

    

    # Computing the balanced logarithmic loss

    balanced_log_loss = (-w0 * log_loss_0 - w1 * log_loss_1)/(w0+w1)

    balanced_log_loss /= 2

    

    return balanced_log_loss

In [8]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, best_score, best_model):
    model.fit(X_train, y_train)
    predictions = model.predict_proba(X_test)
    score = balanced_log_loss(y_test, predictions)
    if score < best_score:
        return score, model
    return best_score, best_model

In [9]:


val_set = StratifiedKFold(n_splits=10, shuffle=True)
# Initialize best models and scores
best_model_lgb, best_model_xg, best_model_cat, best_model_tabpf = None, None, None, None
best_score_lgb, best_score_xg, best_score_cat, best_score_tabpf = float('inf'), float('inf'), float('inf'), float('inf')
for i in range(50, 1001, 50):
    lgb_model = lgb(max_depth=5, learning_rate=1e-2,metric = 'binary_logloss', n_estimators=i, verbose=-1, class_weight="balanced")
    xg_model = xgboost.XGBClassifier(n_estimators = 256, max_depth = 7, scale_pos_weight=scale_pos)
    catboost_model = CatBoostClassifier(iterations=1000, learning_rate=1e-2, depth=5, loss_function='Logloss', verbose=False, auto_class_weights = "Balanced")
    Tabpf_model = tabpfn.TabPFNClassifier(N_ensemble_configurations=64, device='cpu')
    
    for train_index, test_index in val_set.split(train_df, test_df):
        X_train, X_test = train_df[train_index], train_df[test_index]
        y_train, y_test = test_df[train_index], test_df[test_index]
    
        # Evaluate each model using the helper function
        best_score_lgb, best_model_lgb = train_and_evaluate(lgb_model, X_train, y_train, X_test, y_test, best_score_lgb, best_model_lgb)
        best_score_xg, best_model_xg = train_and_evaluate(xg_model, X_train, y_train, X_test, y_test, best_score_xg, best_model_xg)
        best_score_cat, best_model_cat = train_and_evaluate(catboost_model, X_train, y_train, X_test, y_test, best_score_cat, best_model_cat)
        best_score_tabpf, best_model_tabpf = train_and_evaluate(Tabpf_model, X_train, y_train, X_test, y_test, best_score_tabpf, best_model_tabpf)

# Final results
print("Final best LightGBM Model Score:", best_score_lgb)
print("Final best XGBoost Model Score:", best_score_xg)
print("Final best CatBoost Model Score:", best_score_cat)
print("Final best TabPFN Model Score:", best_score_tabpf)
# print("Final best AdaBoost Model Score:", best_score_ada)

Loading model that can be used for inference only


/opt/conda/lib/python3.10/site-packages/tabpfn/scripts/model_builder.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state, optimizer_state, config_sample = torch

Using a Transformer with 25.82 M parameters


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/

Loading model that can be used for inference only


...

Loading model that can be used for inference only


/opt/conda/lib/python3.10/site-packages/tabpfn/scripts/model_builder.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state, optimizer_state, config_sample = torch

Using a Transformer with 25.82 M parameters


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension

Final best LightGBM Model Score: 0.02071280688226751
Final best XGBoost Model Score: 0.026203400030124335
Final best CatBoost Model Score: 0.0314071947245843
Final best TabPFN Model Score: 0.03383185883483464


In [10]:

test_df_pandas = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

scale = StandardScaler().fit(test_df_1)

test_df_1 = scale.transform(test_df_1)

lgb_pred = best_model_lgb.predict_proba(test_df_1)

xg_pred = best_model_xg.predict_proba(test_df_1)

cat_pred = best_model_cat.predict_proba(test_df_1)

tabf_pred = best_model_tabpf.predict_proba(test_df_1)

total_score = best_score_lgb + best_score_xg + best_score_cat + best_score_tabpf

# Calculate total score
total_score = best_score_lgb + best_score_xg + best_score_cat + best_score_tabpf

# Calculate weights for each score
lgb_w = 0.1 # best_score_lgb / total_score
xg_w = 0.1 # best_score_xg / total_score
cat_w =0.25 # best_score_cat / total_score
tabpf_w =0.55 # best_score_tabpf / total_score

# Calculate weighted predictions dynamically
predictions = (lgb_pred * lgb_w +
               xg_pred * xg_w +
               cat_pred * cat_w +
               tabf_pred * tabpf_w)


df = pd.DataFrame({'Id': test_df_pandas.Id, 'class_0': predictions[:,0], 'class_1': predictions[:,1]})

df.to_csv('/kaggle/working/submission.csv', index=False)


/opt/conda/lib/python3.10/site-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map_infer(values, mapper, convert=convert)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/pandas/core/algorithms.py:1743: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  return lib.map